In [5]:
from OperacionesCSV import importar_CSV
from Neo4HungerGames import Neo4HungerGames
from Neo4Dataframes import Neo4Dataframes
import pandas as pd
from flask import Flask, render_template, request
from Controller import Controller

In [6]:
uri = "bolt://localhost:7687"
user = "mdad"
password = "123456789"
neoConnection = Neo4HungerGames(uri, user, password)

In [4]:
ruta_csv='HungerGames_Characters_Dataset_ALL.csv'

hgDF = importar_CSV(ruta_csv)
hgDF['ID'] = hgDF.index
dataConnection = Neo4Dataframes(uri, user, password)
dataConnection.create_nodes_links_from_df(hgDF)
dataConnection.close()

--------------------------------------------


In [7]:
app = Flask(__name__)

# Ruta principal
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/characters', methods=['GET'])
def characters():
    districts = neoConnection.get_all_districts()
    return render_template('characters.html', districts=districts)

@app.route('/characters/all', methods=['GET'])
def all_characters():
    characters = neoConnection.get_all_characters()
    return render_template('characters_list.html', characters=characters)

@app.route('/characters/create', methods=['POST'])
def create_character():
    return Controller.create_character(neoConnection, request=request)

@app.route("/characters/link_game", methods=["GET", "POST"])
def link_character_game():
    return Controller.link_character_game(neoConnection, request=request)

@app.route("/characters/update/<int:character_id>", methods=["GET"])
def edit_character_form(character_id):
    character = neoConnection.get_character_by_id(character_id)
    return render_template("update_character.html", character=character, character_id=character_id)

@app.route("/characters/update/<int:character_id>", methods=["POST"])
def update_character(character_id):
    return Controller.update_character(neoConnection, request, character_id)

@app.route("/characters/delete", methods=["POST"])
def delete_character():
    return Controller.delete_character(neoConnection, request)

@app.route("/characters/delete/<character_id>", methods=["POST"])
def delete_character_by_id(character_id):
    return Controller.delete_character_by_id(neoConnection, character_id)

@app.route("/characters/link_family", methods=["GET"])
def show_family_link_form():
    characters = neoConnection.get_all_characters_id_and_name()
    return render_template("link_family.html", characters=characters)

@app.route("/characters/link_family", methods=["POST"])
def create_family_link():
    return Controller.create_family_link(neoConnection, request)

@app.route("/characters/family/all")
def get_all_family_links():
    return Controller.get_all_family_links(neoConnection)

@app.route("/characters/family/delete", methods=["POST"])
def delete_family_link():
    return Controller.delete_family_link(neoConnection, request.form)

@app.route("/links", methods=["GET"])
def list_links():
    return Controller.list_links(neoConnection)

@app.route("/links/delete", methods=["POST"])
def delete_link():
    return Controller.delete_link(neoConnection, request.form)


In [ ]:
if __name__ == '__main__':
    app.run(debug=True, host="0.0.0.0", port=8080,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.38:8080
Press CTRL+C to quit
127.0.0.1 - - [14/May/2025 01:19:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2025 01:19:18] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [14/May/2025 01:19:21] "GET /characters/family/all HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2025 01:19:21] "GET /static/list.css HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2025 01:19:23] "GET /links HTTP/1.1" 500 -
Traceback (most recent call last):
  File "c:\Users\sarag\anaconda3\Lib\site-packages\flask\app.py", line 1498, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sarag\anaconda3\Lib\site-packages\flask\app.py", line 1476, in wsgi_app
    response = self.handle_exception(e)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sarag\anaconda3\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_di

In [3]:
neoConnection.delete_all_links_and_nodes()
neoConnection.close()